02 Calculate PRS alpha3 AoU participants

Purpose: Calcuate the PRS in alpha3 participants, accounting for mismatches, strand flips and duplicate positions. Also counting the number of indels, mismatches, flips, etc...

using 4 CPUs, 15 GB Ram, 150 GB Disk (Standard VM) Upped to 150 in case there were memory issues



In [1]:
lapply(c('data.table', 'Hmisc', 'bigsnpr','dplyr','stringr'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )
library(data.table)
library(Hmisc)
library(bigsnpr)
library(dplyr)
library(stringr)

Installing package into ‘/home/jupyter/packages’
(as ‘lib’ is unspecified)

also installing the dependencies ‘DEoptimR’, ‘doParallel’, ‘flock’, ‘RhpcBLASctl’, ‘nabor’, ‘robustbase’, ‘rngtools’, ‘iterators’, ‘ff’, ‘bigstatsr’, ‘bigassertr’, ‘bigparallelr’, ‘bigsparser’, ‘bigreadr’, ‘bigutilsr’, ‘doRNG’, ‘foreach’, ‘rmio’




[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: bigstatsr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Hmisc’:

    src, summarize


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
sessionInfo()

R version 4.2.0 (2022-04-22)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] stringr_1.4.0     dplyr_1.0.9       bigsnpr_1.10.8    bigstatsr_1.5.6  
 [5] Hmisc_4.7-1       ggplot2_3.3.6     Formula_1.2-4     survival_3.3-1   
 [9] lattice_0.20-45   data.table_1.14.2

loaded via a namespace (and not attached):
 [1] jsonlite_1

In [3]:
# function to get the complement allele
complement.allele <- function(allele)
{
  out <- ifelse(allele=="A","T",NA)
  out <- ifelse(allele=="T","A",out)
  out <- ifelse(allele=="C","G",out)
  out <- ifelse(allele=="G","C",out)
  out
}
# check that it works
check <- c("A","C","G")
complement.allele(check)

[1] "T" "G" "C"

In [5]:
#describe(prs.total)
rm(prs.total)
for(i in 1:22)
#i <- 22
{
    cat(sprintf("chr %d\n",i)) #write to sink file
    

    # read in files
    # alpha plink files
    infile <- paste0("chr",i)
    cp.command <- paste0("gsutil -m cp -r gs://fc-secure-30fdbdfd-a46b-406d-9617-1bc69ae1da9d/data/alpha3/plink_chr/",infile,".* .")
    system(cp.command, intern=TRUE)
    rm.command1 <- paste0("rm ",infile,".*") #create the command here, but use later
    bedfile <- paste0(infile,".bed")
    rds <- snp_readBed(bedfile,backingfile = tempfile())
    obj.bigSNP <- snp_attach(rds)
    # SNP beta file
    infile <- paste0("ldpred_new_fe2021-chr",i,".b38.txt")
    cp.command <- paste0("gsutil -m cp -r gs://fc-secure-30fdbdfd-a46b-406d-9617-1bc69ae1da9d/",infile," .")
    system(cp.command, intern=TRUE)
    rm.command2 <- paste0("rm ldpred_new_fe2021-chr",i,".b38.txt") #create the command here, but use later

    #get the alpha genotype data
    ID <- obj.bigSNP$fam$sample.ID
    CHR <- obj.bigSNP$map$chromosome
    POS <- obj.bigSNP$map$physical.pos
    G   <- obj.bigSNP$genotypes
    A1 <- obj.bigSNP$map$allele1
    A2 <- obj.bigSNP$map$allele2
    alpha.snps.dt <- data.table(cbind(CHR,POS,A1,A2))
    alpha.snps.dt[,CHR:=str_replace(CHR,"chr","")]
    alpha.snps.dt[,CHR:=as.integer(CHR)]
    alpha.snps.dt[,POS:=as.integer(POS)]

    #initialize prs.total if it doesn't exist yet
   if(!exists("prs.total")){
    prs.total <- rep(0,length(ID))
    cat(sprintf("Initialized PRS total mean is %0.3f\n",mean(prs.total)))
   }

    
    # get the SNP betas
    prs.snps <- fread(infile)
    setnames(prs.snps,names(prs.snps),make.names(names(prs.snps)))
    prs.snps[,CHR:=as.integer(CHR)]
    prs.snps[,POS:=as.integer(POS)]

    #combine SNP info from alpha and beta files
    setkeyv(alpha.snps.dt,c("CHR","POS"))
    setkeyv(prs.snps,c("CHR","POS"))
    betas.dt <- prs.snps[alpha.snps.dt]

    # add in the complement alleles
    betas.dt[,a0c:=complement.allele(a0)]
    betas.dt[,a1c:=complement.allele(a1)]
    
    # zero out the betas for the indels so that they are not included in the analysis
    betas.dt[,num.nucs:=str_length(A1) + str_length(A2)]
    betas.dt[,beta.fix:=ifelse(num.nucs>2,0,beta)]
    cat(sprintf("Number positions with indels %d\n",length(unique(betas.dt$POS[betas.dt$num.nucs>2]))))
    
    # determine which positions have duplicates. some of these will correspond to the indels
    dup.pos <- betas.dt$POS[duplicated(betas.dt$POS)]
    cat(sprintf("Number multiallelic positions (including indels) %d\n",length(unique(dup.pos))))
    cat(sprintf("Number multiallelic positions (excluding indels) %d\n",
                length(setdiff(unique(dup.pos),unique(betas.dt$POS[betas.dt$num.nucs>2])))))
    
    # remove (zero out) the duplicated positions with mismatched alleles. Hopefully there won't be 
    # strand flips that need to be dealt with
    betas.dt[,rm.dup:=ifelse( POS %in% dup.pos & !( (a0==A1 & a1==A2) | (a0==A2 & a1==A1) | 
                                               (a0c==A1 & a1c==A2) | (a0c==A2 & a1c==A1)), 1, 0)]
    betas.dt[,beta.fix:=ifelse(rm.dup==1,0,beta.fix)]
    
    # set up the data tables that go into snp_match(), which corrects the betas automatically
    sumstats <- betas.dt[,list(chr=CHR,pos=POS,a0,a1,beta=beta.fix)]
    info_snp <- betas.dt[,list(chr=CHR,pos=POS,a0=A2,a1=A1)] 
    
    # run the snp_match()
    beta.match.dt <- data.table(snp_match(sumstats,info_snp,return_flip_and_rev=TRUE,remove_dups=FALSE))
    
    #check for SNPs that don't match
    if(length(betas.dt$POS) > length(beta.match.dt$pos))
    { #need to zero out the beta for these SNPs and put back into the data table in correct order
        missing.pos <- setdiff(info_snp$pos,beta.match.dt$pos)
        keep.cols <- c("CHR","POS","A1","A2")
        add.rows <- betas.dt[,.SD,.SDcols=keep.cols][betas.dt$POS %in% missing.pos]
        setnames(add.rows,c("chr","pos","a0","a1"))
        add.rows[,beta:=0]
        add.rows[,"_NUM_ID_.ss":=-1]
        add.rows[,"_FLIP_":=FALSE]
        add.rows[,"_REV_":=FALSE]
        add.rows[,"_NUM_ID_":=-1]
        beta.match.dt <- rbind(add.rows,beta.match.dt)
        setkeyv(beta.match.dt,c("pos","_NUM_ID_.ss"))
    }
    # output number flipped/reversed
    cat(sprintf("Number flipped positions %d\n",length(unique(beta.match.dt$pos[beta.match.dt$"_FLIP_" == TRUE]))))
    cat(sprintf("Number reversed strand positions %d\n",length(unique(beta.match.dt$pos[beta.match.dt$"_REV_" == TRUE]))))    
    cat(sprintf("Number both flipped and reversed positions %d\n",
                length(unique(beta.match.dt$pos[beta.match.dt$"_REV_" == TRUE & beta.match.dt$"_FLIP_" == TRUE]))))
    
    # impute missing genoytpes useing the mode genotype
    G2 <- snp_fastImputeSimple(G,method="mode")

    #calculate the chr. PRS using the betas from beta.match.dt
    prs <- snp_PRS(G=G2, betas.keep = beta.match.dt$beta)[,1]
    prs.dt <- data.table(cbind(ID,prs))
    setnames(prs.dt,c("ID","PRS"))
    cat(sprintf("mean PRS for the chr is %0.3f\n",mean(prs)))

    #output the chr. PRS
    outfile <- paste0("PRS-chr",i,".csv")
    write.csv(prs.dt,outfile,quote=FALSE,row.names=FALSE)
#    cp.command <- paste0("gsutil -m cp -r ",outfile," gs://fc-secure-30fdbdfd-a46b-406d-9617-1bc69ae1da9d/alpha3/")
    cp.command <- paste0("gsutil -m cp -r ",outfile," gs://fc-secure-91b7b957-58d8-4142-b93b-c3a2635f2d78/alpha3/")
    system(cp.command, intern=TRUE)
    
    #calculate total PRS up to this point
    #prs.total <- prs.total + prs.dt$PRS
    cat(sprintf("mean PRS total before adding %0.3f\n",mean(prs.total)))
    prs.total <- prs.total + prs #does this fix my problem
    cat(sprintf("mean PRS total after adding %0.3f\n",mean(prs.total)))
   
    #remove the temporary file
    system(rm.command1, intern=TRUE)
    system(rm.command2, intern=TRUE)
    rm(rds)
    system("rm -r /tmp/Rtmp*")
    
}
#describe(prs.dt$PRS)

#sink()

chr 1


Creating directory "/tmp/RtmpuHeEt4" which didn't exist..



Initialized PRS total mean is 0.000
Number positions with indels 0
Number multiallelic positions (including indels) 11516
Number multiallelic positions (excluding indels) 11516


96,090 variants to be matched.

0 ambiguous SNPs have been removed.

96,090 variants have been matched; 134 were flipped and 347 were reversed.



Number flipped positions 118
Number reversed strand positions 301
Number both flipped and reversed positions 31


...

Number positions with indels 0
Number multiallelic positions (including indels) 2115
Number multiallelic positions (excluding indels) 2115


16,390 variants to be matched.

0 ambiguous SNPs have been removed.

16,390 variants have been matched; 0 were flipped and 219 were reversed.



Number flipped positions 0
Number reversed strand positions 191
Number both flipped and reversed positions 0


'lpS.keep' or 'thr.list' was not specified. Thresholding disabled.



mean PRS for the chr is -0.100
mean PRS total before adding 0.482
mean PRS total after adding 0.382


In [6]:
prs.dt <- data.table(cbind(ID,prs.total))
setnames(prs.dt,c("ID","PRS"))
describe(prs.dt)
outfile <- "total-LDpred.prs.csv"
write.csv(prs.dt,outfile,quote=FALSE,row.names=FALSE)
#cp.command <- paste0("gsutil -m cp -r ", outfile , " gs://fc-secure-30fdbdfd-a46b-406d-9617-1bc69ae1da9d/alpha3/")
cp.command <- paste0("gsutil -m cp -r ", outfile , " gs://fc-secure-91b7b957-58d8-4142-b93b-c3a2635f2d78/alpha3/")
system(cp.command)

prs.dt 

 2  Variables      98622  Observations
--------------------------------------------------------------------------------
ID 
       n  missing distinct     Info     Mean      Gmd      .05      .10 
   98622        0    98622        1  2109763  1007490  1082592  1167547 
     .25      .50      .75      .90      .95 
 1422497  1841526  2560397  3215596  3431237 

lowest : 1000004 1000033 1000042 1000091 1000095
highest: 9991636 9992998 9993207 9995407 9997756
--------------------------------------------------------------------------------
PRS 
       n  missing distinct     Info     Mean      Gmd      .05      .10 
   98622        0    98622        1   0.3824   0.5724 -0.45546 -0.26598 
     .25      .50      .75      .90      .95 
 0.04131  0.38635  0.72714  1.03091  1.21179 

lowest : -1.982690 -1.849360 -1.756135 -1.750989 -1.721825
highest:  2.400490  2.406779  2.477466  2.499369  2.671126
--------------------------------------------------------------------------------

In [ ]:
head(prs.dt)
hist(prs.dt$PRS)
